# Artistic QR Code Generation on Kaggle

This notebook generates artistic images with embedded scannable QR codes using Stable Diffusion and ControlNet.

## Features
- Generate artistic images with embedded QR codes
- Support for both ControlNet and post-processing embedding methods
- Automatic QR code validation
- GPU acceleration support


In [ ]:
# Install required packages
!pip install -q diffusers>=0.21.0 transformers>=4.30.0 accelerate>=0.20.0
!pip install -q qrcode[pil]>=7.4.2 Pillow>=10.0.0
!pip install -q pyzbar opencv-python
!pip install -q scipy  # Optional, for edge enhancement


In [ ]:
# Import libraries
import os
import qrcode
import torch
import numpy as np
from PIL import Image
from diffusers import StableDiffusionPipeline, StableDiffusionControlNetPipeline, ControlNetModel
from typing import Optional
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


In [ ]:
# Kaggle-specific paths
KAGGLE_WORKING = "/kaggle/working"
KAGGLE_INPUT = "/kaggle/input"
OUTPUT_DIR = os.path.join(KAGGLE_WORKING, "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Set up Hugging Face cache (use working directory)
HF_CACHE = os.path.join(KAGGLE_WORKING, "hf_cache")
os.makedirs(HF_CACHE, exist_ok=True)
os.environ["HF_HOME"] = HF_CACHE
os.environ["HF_HUB_CACHE"] = os.path.join(HF_CACHE, "hub")

print(f"Output directory: {OUTPUT_DIR}")
print(f"HF Cache: {HF_CACHE}")


In [ ]:
# Artistic QR Pipeline Class (Kaggle-adapted)
class ArtisticQRPipeline:
    """Complete pipeline for generating artistic images with embedded QR codes."""
    
    def __init__(self, 
                 model_id: str = "runwayml/stable-diffusion-v1-5",
                 cache_dir: str = None,
                 device: Optional[str] = None):
        self.model_id = model_id
        self.cache_dir = cache_dir or HF_CACHE
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.pipe = None
        self.controlnet_pipe = None
        self.controlnet_model = None
        
    def create_qr_code(self, data: str, size: int = 512, border: int = 4, 
                       error_correction: str = "H") -> Image.Image:
        """Create a QR code with specified parameters."""
        error_levels = {
            'L': qrcode.constants.ERROR_CORRECT_L,
            'M': qrcode.constants.ERROR_CORRECT_M,
            'Q': qrcode.constants.ERROR_CORRECT_Q,
            'H': qrcode.constants.ERROR_CORRECT_H
        }
        
        qr = qrcode.QRCode(
            version=1,
            error_correction=error_levels.get(error_correction.upper(), qrcode.constants.ERROR_CORRECT_H),
            box_size=10,
            border=border,
        )
        qr.add_data(data)
        qr.make(fit=True)
        
        qr_img = qr.make_image(fill_color="black", back_color="white")
        qr_img = qr_img.resize((size, size), Image.Resampling.LANCZOS)
        return qr_img
    
    def load_model(self):
        """Load the Stable Diffusion model."""
        if self.pipe is not None:
            return self.pipe
        
        print(f"Loading Stable Diffusion model: {self.model_id}")
        dtype = torch.float16 if self.device == "cuda" else torch.float32
        
        self.pipe = StableDiffusionPipeline.from_pretrained(
            self.model_id,
            torch_dtype=dtype,
            cache_dir=self.cache_dir
        ).to(self.device)
        
        if self.device == "cuda":
            try:
                self.pipe.enable_xformers_memory_efficient_attention()
            except:
                print("xformers not available, continuing without it")
        
        print("Model loaded successfully!")
        return self.pipe
    
    def load_controlnet_model(self):
        """Load ControlNet model for QR code generation."""
        if self.controlnet_pipe is not None:
            return self.controlnet_pipe
        
        print("Loading ControlNet QR code model...")
        dtype = torch.float16 if self.device == "cuda" else torch.float32
        
        # Try different ControlNet models
        controlnet_models = [
            "monster-labs/control_v1p_sd15_qrcode_monster",
            "DionTimmer/controlnet_qrcode-control_v11p_sd15",
        ]
        
        controlnet = None
        for model_id in controlnet_models:
            try:
                print(f"Trying ControlNet model: {model_id}")
                controlnet = ControlNetModel.from_pretrained(
                    model_id,
                    torch_dtype=dtype,
                    cache_dir=self.cache_dir
                )
                print(f"Successfully loaded: {model_id}")
                self.controlnet_model = model_id
                break
            except Exception as e:
                print(f"Failed to load {model_id}: {e}")
                continue
        
        if controlnet is None:
            raise RuntimeError("Could not load any ControlNet QR code model")
        
        self.controlnet_pipe = StableDiffusionControlNetPipeline.from_pretrained(
            self.model_id,
            controlnet=controlnet,
            torch_dtype=dtype,
            cache_dir=self.cache_dir,
            safety_checker=None,
            requires_safety_checker=False
        ).to(self.device)
        
        if self.device == "cuda":
            try:
                self.controlnet_pipe.enable_xformers_memory_efficient_attention()
            except:
                print("xformers not available, continuing without it")
        
        print("ControlNet model loaded successfully!")
        return self.controlnet_pipe


In [ ]:
# Continue with embedding methods
def embed_qr_artistically(base_image, qr_image, subtlety=0.88, contrast_boost=0.08):
    """Embed QR code into image artistically."""
    if base_image.mode != 'RGB':
        base_image = base_image.convert('RGB')
    if qr_image.mode != 'RGB':
        qr_image = qr_image.convert('RGB')
    
    target_size = max(base_image.size)
    base_image = base_image.resize((target_size, target_size), Image.Resampling.LANCZOS)
    qr_image = qr_image.resize((target_size, target_size), Image.Resampling.LANCZOS)
    
    base_array = np.array(base_image, dtype=float)
    qr_array = np.array(qr_image, dtype=float)
    
    # Convert QR code to binary mask
    qr_gray = np.dot(qr_array[...,:3], [0.2989, 0.5870, 0.1140])
    threshold = 127
    qr_binary = (qr_gray < threshold).astype(float)
    qr_mask_3d = np.expand_dims(qr_binary, axis=2)
    
    # Calculate contrast
    min_contrast = 0.20
    if subtlety <= 0.85:
        effective_contrast = 0.25
    elif subtlety <= 0.88:
        effective_contrast = 0.25 - (subtlety - 0.85) * (0.25 - 0.22) / (0.88 - 0.85)
    elif subtlety <= 0.90:
        effective_contrast = 0.22 - (subtlety - 0.88) * (0.22 - 0.20) / (0.90 - 0.88)
    else:
        effective_contrast = min_contrast
    effective_contrast = max(effective_contrast, min_contrast)
    
    dark_factor = 1.0 - effective_contrast
    light_factor = 1.0 + effective_contrast
    
    result_array = base_array.copy()
    
    # Apply embedding
    dark_mask = qr_mask_3d
    result_array = result_array * (1 - dark_mask * (1 - dark_factor))
    
    light_mask = 1 - qr_mask_3d
    result_array = result_array * (1 + light_mask * (light_factor - 1))
    
    # Contrast boost
    if contrast_boost > 0:
        qr_contrast = (qr_mask_3d - 0.5) * 2
        contrast_multiplier = 100 if subtlety >= 0.88 else 80
        contrast_strength = contrast_boost * contrast_multiplier
        result_array = result_array + (qr_contrast * contrast_strength)
    
    # Pattern enhancement
    qr_pattern_diff = (qr_mask_3d - 0.5) * 2
    pattern_enhancement = 15.0
    result_array = result_array + (qr_pattern_diff * pattern_enhancement)
    
    result_array = np.clip(result_array, 0, 255).astype(np.uint8)
    return Image.fromarray(result_array)


In [ ]:
# Add methods to pipeline class
def generate_with_controlnet(self, prompt, qr_data, output_path, image_size=512,
                             num_inference_steps=30, guidance_scale=7.5,
                             controlnet_conditioning_scale=1.5, seed=None,
                             qr_enhancement_strength=0.20):
    """Generate artistic QR code using ControlNet."""
    print("=" * 60)
    print("ControlNet Artistic QR Code Generation")
    print("=" * 60)
    
    # Create QR code
    print("\n[Step 1/2] Creating QR code...")
    qr_image = self.create_qr_code(qr_data, size=image_size)
    qr_image = qr_image.convert("RGB")
    
    # Save original QR
    qr_ref_path = output_path.replace('.png', '_original_qr.png')
    qr_image.save(qr_ref_path)
    print(f"Original QR code saved: {qr_ref_path}")
    
    # Load ControlNet
    print("\n[Step 2/2] Generating image with ControlNet...")
    pipe = self.load_controlnet_model()
    
    generator = None
    if seed is not None:
        generator = torch.Generator(device=self.device).manual_seed(seed)
    
    image = pipe(
        prompt=prompt,
        negative_prompt="blurry, distorted, unreadable qr, low quality",
        image=qr_image,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        controlnet_conditioning_scale=controlnet_conditioning_scale,
        generator=generator
    ).images[0]
    
    # Enhance QR scannability
    print("\n[Step 3/3] Enhancing QR code scannability...")
    image = enhance_qr_scannability(image, qr_image, qr_enhancement_strength)
    
    image.save(output_path)
    print(f"\n✓ ControlNet artistic QR code saved to: {output_path}")
    return image

def enhance_qr_scannability(generated_image, qr_reference, enhancement_strength=0.20):
    """Enhance QR code scannability in ControlNet-generated images."""
    if generated_image.mode != 'RGB':
        generated_image = generated_image.convert('RGB')
    if qr_reference.mode != 'RGB':
        qr_reference = qr_reference.convert('RGB')
    
    target_size = generated_image.size[0]
    qr_reference = qr_reference.resize((target_size, target_size), Image.Resampling.LANCZOS)
    
    gen_array = np.array(generated_image, dtype=float)
    qr_array = np.array(qr_reference, dtype=float)
    
    qr_gray = np.dot(qr_array[...,:3], [0.2989, 0.5870, 0.1140])
    threshold = 127
    qr_binary = (qr_gray < threshold).astype(float)
    qr_mask_3d = np.expand_dims(qr_binary, axis=2)
    
    result_array = gen_array.copy()
    
    dark_mask = qr_mask_3d
    darken_factor = 1.0 - (enhancement_strength * 0.5)
    result_array = result_array * (1 - dark_mask * (1 - darken_factor))
    
    light_mask = 1 - qr_mask_3d
    lighten_factor = 1.0 + (enhancement_strength * 0.5)
    result_array = result_array * (1 + light_mask * (lighten_factor - 1))
    
    contrast_boost = enhancement_strength * 30
    qr_contrast = (qr_mask_3d - 0.5) * 2
    result_array = result_array + (qr_contrast * contrast_boost)
    
    result_array = np.clip(result_array, 0, 255).astype(np.uint8)
    return Image.fromarray(result_array)

# Attach methods to class
ArtisticQRPipeline.generate_with_controlnet = generate_with_controlnet


In [ ]:
# Complete pipeline method
def process(self, prompt, qr_data, output_path, image_size=512, subtlety=0.88,
           num_inference_steps=50, guidance_scale=7.5, seed=None,
           use_controlnet=False, controlnet_conditioning_scale=1.5,
           qr_enhancement_strength=0.20):
    """Complete pipeline: Generate image and embed QR code."""
    
    if use_controlnet:
        return self.generate_with_controlnet(
            prompt=prompt, qr_data=qr_data, output_path=output_path,
            image_size=image_size, num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale, controlnet_conditioning_scale=controlnet_conditioning_scale,
            seed=seed, qr_enhancement_strength=qr_enhancement_strength
        )
    
    # Post-processing embedding method
    print("=" * 60)
    print("Artistic QR Code Image Generation Pipeline")
    print("=" * 60)
    
    # Step 1: Create QR code
    print("\n[Step 1/3] Creating QR code...")
    qr_image = self.create_qr_code(qr_data, size=image_size)
    
    qr_ref_path = output_path.replace('.png', '_original_qr.png')
    qr_image.save(qr_ref_path)
    print(f"Original QR code saved: {qr_ref_path}")
    
    # Step 2: Generate artistic image
    print("\n[Step 2/3] Generating artistic image...")
    if self.pipe is None:
        self.load_model()
    
    generator = None
    if seed is not None:
        generator = torch.Generator(device=self.device).manual_seed(seed)
    
    generated_image = self.pipe(
        prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator
    ).images[0]
    
    generated_image = generated_image.resize((image_size, image_size), Image.Resampling.LANCZOS)
    
    # Step 3: Embed QR code
    print("\n[Step 3/3] Embedding QR code artistically...")
    contrast_boost_value = 0.08 if subtlety >= 0.90 else 0.06
    final_image = embed_qr_artistically(
        generated_image, qr_image, subtlety=subtlety, contrast_boost=contrast_boost_value
    )
    
    final_image.save(output_path)
    print(f"\n✓ Artistic QR code image saved to: {output_path}")
    return final_image

ArtisticQRPipeline.process = process


## Generate Artistic QR Code

Now let's generate an artistic QR code image. You can customize the parameters below.


In [ ]:
# Configuration
PROMPT = "a king looking up at the sky, white fluffy clouds in blue sky, beautiful sunset colors, artistic illustration, detailed fur texture, expressive eyes, QR code pattern subtly integrated into clouds and sky background, high quality, vibrant colors"
QR_DATA = "https://example.com"
IMAGE_SIZE = 512
USE_CONTROLNET = True  # Set to False for post-processing embedding
SEED = 42

# ControlNet parameters (only used if USE_CONTROLNET=True)
CONTROLNET_CONDITIONING_SCALE = 1.5
QR_ENHANCEMENT_STRENGTH = 0.20

# Post-processing parameters (only used if USE_CONTROLNET=False)
SUBTLETY = 0.88

# Generation parameters
NUM_INFERENCE_STEPS = 30 if USE_CONTROLNET else 50
GUIDANCE_SCALE = 7.5

print("Configuration:")
print(f"  Prompt: {PROMPT[:50]}...")
print(f"  QR Data: {QR_DATA}")
print(f"  Method: {'ControlNet' if USE_CONTROLNET else 'Post-processing Embedding'}")
print(f"  Image Size: {IMAGE_SIZE}")
print(f"  Seed: {SEED}")


In [ ]:
# Initialize pipeline
pipeline = ArtisticQRPipeline(
    model_id="runwayml/stable-diffusion-v1-5",
    cache_dir=HF_CACHE,
    device=device
)

# Generate QR code
output_filename = "artistic_qr_output.png"
output_path = os.path.join(OUTPUT_DIR, output_filename)

print("\nStarting generation...")
final_image = pipeline.process(
    prompt=PROMPT,
    qr_data=QR_DATA,
    output_path=output_path,
    image_size=IMAGE_SIZE,
    subtlety=SUBTLETY,
    num_inference_steps=NUM_INFERENCE_STEPS,
    guidance_scale=GUIDANCE_SCALE,
    seed=SEED,
    use_controlnet=USE_CONTROLNET,
    controlnet_conditioning_scale=CONTROLNET_CONDITIONING_SCALE,
    qr_enhancement_strength=QR_ENHANCEMENT_STRENGTH
)

print(f"\n✅ Generation complete!")
print(f"Output saved to: {output_path}")


In [ ]:
# Display the generated image
from IPython.display import Image as IPImage, display

display(IPImage(output_path, width=512))
print(f"\nGenerated QR Code Image")
print(f"QR Data: {QR_DATA}")

# Also display original QR for comparison
qr_ref_path = output_path.replace('.png', '_original_qr.png')
if os.path.exists(qr_ref_path):
    print("\nOriginal QR Code (for comparison):")
    display(IPImage(qr_ref_path, width=256))


In [ ]:
# Optional: Validate QR code scannability
try:
    from pyzbar.pyzbar import decode as pyzbar_decode
    import cv2
    
    def validate_qr(image_path, expected_data=None):
        """Validate QR code scannability."""
        img = cv2.imread(image_path)
        if img is None:
            return {"scannable": False, "error": "Could not read image"}
        
        # Try OpenCV QRCodeDetector
        detector = cv2.QRCodeDetector()
        retval, decoded_info, points, straight_qrcode = detector.detectAndDecodeMulti(img)
        
        if retval and decoded_info:
            for info in decoded_info:
                if info:
                    matches = (info == expected_data) if expected_data else True
                    return {
                        "scannable": True,
                        "data_decoded": info,
                        "matches_expected": matches
                    }
        
        # Try PyZBar
        pil_img = Image.open(image_path)
        img_array = np.array(pil_img.convert('RGB'))
        decoded_objects = pyzbar_decode(img_array)
        
        if decoded_objects:
            for obj in decoded_objects:
                if obj.type == 'QRCODE':
                    decoded = obj.data.decode('utf-8')
                    matches = (decoded == expected_data) if expected_data else True
                    return {
                        "scannable": True,
                        "data_decoded": decoded,
                        "matches_expected": matches
                    }
        
        return {"scannable": False, "error": "Could not decode QR code"}
    
    # Validate
    print("Validating QR code scannability...")
    result = validate_qr(output_path, QR_DATA)
    
    print("\n" + "="*60)
    print("QR Code Validation Results")
    print("="*60)
    print(f"Scannable: {'✅ YES' if result['scannable'] else '❌ NO'}")
    if result.get('data_decoded'):
        print(f"Decoded Data: {result['data_decoded']}")
        if 'matches_expected' in result:
            print(f"Matches Expected: {'✅ YES' if result['matches_expected'] else '❌ NO'}")
    if result.get('error'):
        print(f"Error: {result['error']}")
    print("="*60)
    
except ImportError:
    print("⚠️  QR validation libraries not available. Install pyzbar and opencv-python for validation.")
except Exception as e:
    print(f"⚠️  Validation error: {e}")


## Notes

- **Output files** are saved to `/kaggle/working/outputs/`
- **Models** are cached in `/kaggle/working/hf_cache/`
- **GPU** is automatically detected and used if available
- **ControlNet** method generally produces more scannable QR codes
- Adjust `subtlety` (0.85-0.95) for post-processing method - lower = more visible QR
- Adjust `qr_enhancement_strength` (0.15-0.25) for ControlNet method - higher = more visible QR

## Tips for Better Scannability

1. **Use ControlNet** (`use_controlnet=True`) for better results
2. **Lower subtlety** (0.85-0.88) for post-processing method
3. **Higher enhancement strength** (0.20-0.25) for ControlNet method
4. **Test with phone camera** - sometimes works even if validation fails
